In [47]:
#dependencies
import pandas as pd
import json
import datetime
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

In [48]:
us_videos = pd.read_csv('USvideos.csv')
us_videos_categories = pd.read_json('US_category_id.json')


In [49]:
us_videos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                40949 non-null  object
 1   trending_date           40949 non-null  object
 2   title                   40949 non-null  object
 3   channel_title           40949 non-null  object
 4   category_id             40949 non-null  int64 
 5   publish_time            40949 non-null  object
 6   tags                    40949 non-null  object
 7   views                   40949 non-null  int64 
 8   likes                   40949 non-null  int64 
 9   dislikes                40949 non-null  int64 
 10  comment_count           40949 non-null  int64 
 11  thumbnail_link          40949 non-null  object
 12  comments_disabled       40949 non-null  bool  
 13  ratings_disabled        40949 non-null  bool  
 14  video_error_or_removed  40949 non-null  bool  
 15  de

In [50]:
us_videos['trending_date'] = pd.to_datetime(us_videos['trending_date'], format='%y.%d.%m').dt.date

In [51]:
publish_time = pd.to_datetime(us_videos.publish_time, format='%Y-%m-%dT%H:%M:%S.%fZ')

# Create Variable publish_date
us_videos['publish_date'] = publish_time.dt.date

# Drop publish_time
us_videos.drop('publish_time',axis=1,inplace=True)

In [52]:
columns =  ['video_id' ,
	'trending_date' ,
	'title' ,
	'channel_title' ,	
	'category_id' ,
	'publish_time' ,
	'tags' ,
	'views' ,
	'likes' ,
	'dislikes' ,	
	'comment_count']

In [53]:
usvid_df = pd.read_csv('USvideos.csv')
usvid_df.head()


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [54]:
drcolumns = ['thumbnail_link',	'comments_disabled',	'ratings_disabled',	'video_error_or_removed', 'description']
usvid_df.drop(drcolumns, inplace=True, axis=1)

usvid_df.head()



,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518


In [55]:
new_USV_df = pd.DataFrame(usvid_df.tags.str.split('|').tolist(), index=usvid_df.video_id).stack()
new_USV_df.head()

video_id      
2kyS6SvSYSE  0                       SHANtell martin
1ZAPwfrtAFY  0    last week tonight trump presidency
             1      "last week tonight donald trump"
             2                   "john oliver trump"
             3                        "donald trump"
dtype: object

In [57]:
new_USV_df

video_id       
2kyS6SvSYSE  0                        SHANtell martin
1ZAPwfrtAFY  0     last week tonight trump presidency
             1       "last week tonight donald trump"
             2                    "john oliver trump"
             3                         "donald trump"
                                  ...                
oV0zkMe1K8s  21                    "ending explained"
ooyjaVdt-jA  0                           call of duty
             1                                  "cod"
             2                           "activision"
             3                          "Black Ops 4"
Length: 808183, dtype: object

In [58]:
rds_connection_string = "postgres:norocel7@database-1.cgiopwhezyby.us-east-2.rds.amazonaws.com/yt_data"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [59]:
new_USV_df.to_sql('yt_2nf', engine)

ValueError: Table 'yt_2nf' already exists.